In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [71]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiSeriesCustom import ForecasterAutoregMultiSeriesCustom
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries

# Fixtures
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import exog
series.index = pd.date_range(start='2024-01-01', periods=len(series), freq='D')
exog.index = pd.date_range(start='2024-01-01', periods=len(exog), freq='D')
series_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection_multiseries\tests\fixture_sample_multi_series.joblib")
exog_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection_multiseries\tests\fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}
series_with_nans = series.copy()
series_with_nans.iloc[:10, series_with_nans.columns.get_loc('l2')] = np.nan

In [74]:
    forecaster = ForecasterAutoregMultiVariate(
                     regressor          = Ridge(random_state=123),
                     level              = 'l1',
                     lags               = 2,
                     steps              = 2,
                     transformer_series = None
                 )
    
    refit = 2
    n_jobs = 2
    n_validation = 20
    steps = 2

    warn_msg = re.escape(
        ("If `refit` is an integer other than 1 (intermittent refit). `n_jobs` "
         "is set to 1 to avoid unexpected results during parallelization.")
    )
    with pytest.warns(IgnoredArgumentWarning, match = warn_msg):
        metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                                   forecaster              = forecaster,
                                                   series                  = series,
                                                   steps                   = steps,
                                                   levels                  = 'l1',
                                                   metric                  = 'mean_absolute_error',
                                                   add_aggregated_metric   = False,
                                                   initial_train_size      = len(series) - n_validation,
                                                   gap                     = 0,
                                                   allow_incomplete_fold   = False,
                                                   refit                   = refit,
                                                   fixed_train_size        = True,
                                                   exog                    = series['l1'].rename('exog_1'),
                                                   interval                = [5, 95],
                                                   n_boot                  = 100,
                                                   random_state            = 123,
                                                   use_in_sample_residuals = True,
                                                   verbose                 = False,
                                                   n_jobs                  = n_jobs
                                               )

  0%|          | 0/10 [00:00<?, ?it/s]

In [76]:
series

,l1,l2
2024-01-01,0.696469,0.120629
2024-01-02,0.286139,0.826341
2024-01-03,0.226851,0.603060
2024-01-04,0.551315,0.545068
2024-01-05,0.719469,0.342764
2024-01-06,0.423106,0.304121
2024-01-07,0.980764,0.417022
2024-01-08,0.684830,0.681301
2024-01-09,0.480932,0.875457
2024-01-10,0.392118,0.510422


In [75]:
backtest_predictions

,l1,l1_lower_bound,l1_upper_bound
2024-01-31,0.298326,0.148141,0.526117
2024-02-01,0.453676,0.277323,0.560421
2024-02-02,0.480286,0.330100,0.708077
2024-02-03,0.483848,0.307496,0.590594
2024-02-04,0.449913,0.274526,0.656724
2024-02-05,0.380623,0.209464,0.510767
2024-02-06,0.428174,0.252787,0.634985
2024-02-07,0.682095,0.510936,0.812239
2024-02-08,0.727866,0.588433,0.865294
2024-02-09,0.451477,0.328638,0.627053


In [65]:
backtest_predictions.to_numpy()

array([[0.53203113, 0.27318916, 0.69931226],
       [0.62129188, 0.46220068, 0.85870486],
       [0.40145922, 0.17189084, 0.59797159],
       [0.38821275, 0.17098498, 0.61456487],
       [0.37636995, 0.11752798, 0.54365108],
       [0.36599243, 0.20690123, 0.60340541],
       [0.47654862, 0.24698023, 0.67306099],
       [0.33725755, 0.12002978, 0.56360967],
       [0.46122163, 0.20237966, 0.62850276],
       [0.47541934, 0.31632814, 0.71283232],
       [0.50601999, 0.27645161, 0.70253236],
       [0.39293452, 0.17570675, 0.61928664],
       [0.43793405, 0.27695885, 0.5990429 ],
       [0.52600886, 0.34911522, 0.64736209],
       [0.70882239, 0.55319058, 0.83377003],
       [0.70063755, 0.53323585, 0.85415566],
       [0.48631217, 0.32533698, 0.64742102],
       [0.52248417, 0.34559054, 0.6438374 ],
       [0.30010035, 0.14446853, 0.42504798],
       [0.43342987, 0.26602816, 0.58694797],
       [0.42479569, 0.2638205 , 0.58590454],
       [0.68566159, 0.50876795, 0.80701482],
       [0.

In [53]:
backtest_predictions.index

DatetimeIndex(['2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06',
               '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10',
               '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14',
               '2022-02-15', '2022-02-16', '2022-02-17'],
              dtype='datetime64[ns]', freq='D')